<a href="https://colab.research.google.com/github/anmolkumar0717/pyTorch/blob/main/01PyTorch_workFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Python workflow


In [ ]:
what_we_covering={1:"Load a data or prepare",
                  2:"build model",
                  3:"fitting the data to the model",
                  4:"making prediction and evaluating the model(inference)",
                  5:"saving and loading a model",
                  6:"putting all together"}

In [4]:
import torch
from torch import nn #nn is the building block of the nueral network
import matplotlib.pyplot as plt
torch.__version__


'2.2.1+cu121'

#1... Data (preapearing and loading)

##data can be anything ......

####1.Excel spreadsheet...
####2.videos (yt have a lot of data)
####3.images of any kind
####4.Audios of poadcast
####5.DNA
####6.Text

#machine learning is all about two parts:

####1.Get the data into numerical form
####2.Build the model to learn its pattern in numerical representation


##To showcase it lets create some *known* data using linear regression formula

we use linear regression formula to make a stright line with some known *parameters* data

In [5]:
#create a *known* parameters

weight=0.7
bias= 0.3

#create

start=0
end=1
step=0.03
X= torch.arange(start,end,step)
Y= weight*X + bias

X[:10],Y[:10],len(X),len(Y)


(tensor([0.0000, 0.0300, 0.0600, 0.0900, 0.1200, 0.1500, 0.1800, 0.2100, 0.2400,
         0.2700]),
 tensor([0.3000, 0.3210, 0.3420, 0.3630, 0.3840, 0.4050, 0.4260, 0.4470, 0.4680,
         0.4890]),
 34,
 34)